In [ ]:
# !pip install vaderSentiment
import tweepy
import json
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress

#Import sentiment analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from api_keys import consumer_key, consumer_secret, access_token, access_token_secret
#
analyzer = SentimentIntensityAnalyzer()

In [ ]:
#Instantiate Authentication / Wrapper
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
#Observe 5 different Twitter accounts from Major Media
handles = ['BBC', 'CBSNews', 'CNN', 'FoxNews', 'nytimes']

In [ ]:
#Create Empty Lists for Tweets
tweets = []
name = []
full_text = []

In [ ]:
#Loop through handles and compile last 100 tweets from each
for i in handles:
    tweets = tweepy.Cursor(api.user_timeline, i, tweet_mode='extended').items(100)
    for j in tweets:
        name.append(j.author.name)
        full_text.append(j.full_text)

In [ ]:
#Create DataFrame with Tweets
twitter_df = pd.DataFrame({'Name' : name, 'Tweet Text' : full_text})
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
twitter_df

In [ ]:
#Give each tweet an index # that resets once the name of the handle changes
base_name = twitter_df.loc[0,'Name']
counter = 0 
for index, row in twitter_df.iterrows():
    if row['Name'] == base_name:
        twitter_df.loc[index,'Tweets Ago'] = counter
        counter +=1
    if row['Name'] != base_name:
        counter = 0
        twitter_df.loc[index,'Tweets Ago'] = counter
        base_name = row['Name']
        counter +=1

In [ ]:
#Analyze the Sentiment of each Tweet
for index, row in twitter_df.iterrows():
    tweet = row['Tweet Text']
    results = analyzer.polarity_scores(f'{tweet}')
    twitter_df.loc[index, 'Compound'] = results['compound']
    twitter_df.loc[index, 'Positive'] = results['pos']
    twitter_df.loc[index, 'Neutral'] = results['neu']
    twitter_df.loc[index, 'Negative'] = results['neg']
twitter_df.head()

In [ ]:
#Create Plot Points for Graphs
x = twitter_df.iloc[0:100,2]
#
bbc = twitter_df.loc[twitter_df['Name']=='BBC']['Compound']
cnn = twitter_df.loc[twitter_df['Name']=='CNN']['Compound']
cbs = twitter_df.loc[twitter_df['Name']== 'CBS News']['Compound']
fox = twitter_df.loc[twitter_df['Name']=='Fox News']['Compound']
nyt = twitter_df.loc[twitter_df['Name']=='The New York Times']['Compound']

In [ ]:
plt.scatter(x,bbc, color = 'blue')
plt.scatter(x,cnn, color = 'green')
plt.scatter(x,cbs, color = 'orange')
plt.scatter(x,fox, color = 'red')
plt.scatter(x,nyt, color = 'purple')
plt.xticks(rotation = 45)
plt.title('Sentiment Analysis of Major Media Outlet Tweets')
plt.xlabel("Tweets Ago")
plt.ylabel("Compound Score")
plt.grid(True)
plt.legend()
plt.savefig('output/tweepy/SentimentbyTweet')

In [ ]:
polarity = twitter_df.groupby(['Name']).mean()["Compound"]
pd.DataFrame(polarity)

In [ ]:
fig, ax = plt.subplots()
x = np.arange(len(polarity))
bbc = ax.bar(x[0], polarity[0], color = 'blue')
cbs = ax.bar(x[1], polarity[1], color = 'orange')
cnn = ax.bar(x[2], polarity[2], color = 'green')
fox = ax.bar(x[3], polarity[3], color = 'red')
nyt = ax.bar(x[4], polarity[4], color = 'purple')
ax.set_ylabel("Tweet Polarity")
ax.set_title("Overall Media Sentiment based on Twitter")
ax.set_xticks(x + 0.5)
ax.set_xticklabels(("BBC", "CBS", "CNN", "Fox", "NYT"))
ax.grid(False)
plt.savefig('output/tweepy/OverallSentiment')

In [ ]:
#FoxNews does NOT put text in their tweets - only links to articles. Therefore VADER cannot assign a score